In [1]:
from manim import *
from manim_extensions import *
from circuits.circuit_mobjects import *
import circuits.theoretical as tl
from random import random

Manim Community v0.15.2

In [2]:
def fancy(mob, length=1, animation=DrawBorderThenFill, rate_func=None, *args, **kwargs):
    return AnimationGroup(*[
        animation(mob[i], *args, **occur_at(i / len(mob) * length, (len(mob) - i) / len(mob) * length, length, rate_func=rate_func), introducer=False, **kwargs)
        # animation(mob[i], run_time=(len(mob) - i) / len(mob) * length, lag_ratio=i / len(mob))
        for i in range(len(mob))
    ])

def consecutive(mob, length, animation=DrawBorderThenFill, rate_func=None, *args, **kwargs):
    return AnimationGroup(*[
        animation(mob[i], *args, **occur_at(i / len(mob) * length, 1 / len(mob) * length, length, rate_func=rate_func), introducer=False, **kwargs)
        for i in range(len(mob))
    ])


In [16]:
%%manim -v WARNING -qm SVG1

class SVG1(Scene):
    def construct(self):
        computers_and_phones = SVGMobject("./svgs/computers_and_phones.svg").scale(3)
        satellite_communication = SVGMobject("./svgs/satellite_communication.svg").scale(3)
        synapses_brain = SVGMobject("./svgs/synapses_brain.svg").scale(3)
        heart = SVGMobject("./svgs/heart.svg").scale(2)
        power_lines_usa = SVGMobject("./svgs/power_lines_usa.svg").scale(3).shift(RIGHT)
        
        # Create computers_and_phones
        x = ValueTracker(0)
        self.play(*fancy(computers_and_phones, 2))
        self.clear()
        computers_and_phones_dup = always_redraw(
            lambda: computers_and_phones
                .copy()
                .scale(1 / (x.get_value() + 1))
                .move_to(x.get_value() * DOWN)
        )
        satellite_communication_dup = always_redraw(
            lambda: satellite_communication
                .copy()
                .scale(1 / x.get_value())
        )
        self.add(computers_and_phones_dup)

        self.wait(2)

        # Drop computers_and_phones
        self.play(x.animate.set_value(5), run_time=1)
        self.clear()
        x.set_value(0)

        # Create satellite_communication
        self.add(satellite_communication_dup)
        self.play(x.animate.set_value(1), run_time=2)

        self.play(FadeOut(satellite_communication_dup))

        # Create brain
        num_electricities = 9
        self.play(Create(synapses_brain[:-num_electricities]), run_time=2)
        self.play(*consecutive(synapses_brain[-num_electricities:], 2, animation=FadeIn, rate_func=lambda a: there_and_back((a - 0.5) * 2 + 0.5)))
        self.play(FadeOut(synapses_brain))

        # Create heart
        self.play(*fancy(heart, 1))
        for _ in range(4):
            self.play(
                heart.animate.scale(1.2), 
                rate_func=lambda x: there_and_back(2*x), run_time=1
            )
        self.remove(heart)

        # Create power lines
        self.add(power_lines_usa)
        nodes = 7
        DALLAS, ATLANTA, NEW_YORK, MINNEAPOLIS, SEATTLE, LAS_VAGAS, LAS_ANGELES = range(nodes)
        circuit = tl.ACCircuit(nodes)
        voltage_sources = [
            (DALLAS, LAS_VAGAS),
            (LAS_ANGELES, SEATTLE),
            (MINNEAPOLIS, LAS_VAGAS),
            (ATLANTA, NEW_YORK),
        ]
        resistors = [
            (DALLAS, ATLANTA),
            (ATLANTA, MINNEAPOLIS),
            (NEW_YORK, MINNEAPOLIS),
            (MINNEAPOLIS, DALLAS),
            (DALLAS, LAS_ANGELES),
            (LAS_ANGELES, LAS_VAGAS),
            (LAS_VAGAS, SEATTLE),
        ]
        [circuit.add(tl.IndependantVoltage(3 * random(), *vs)) for vs in voltage_sources]
        [circuit.add(tl.Resistor(1, *r)) for r in resistors]

        circuit.nodal_analysis()
        circuit.calculate_currents()

        cmob_circuit = circuit.get_mobjects(
            np.concatenate(([
                [0, -1.2],  # DALLAS
                [2.8, -1.4],# ATLANTA
                [4, 1],     # NEW_YORK
                [0.5, 2],   # MINNEAPOLIS
                [-4, 2.5],  # SEATTLE
                [-3, 0.5],  # LAS_VAGAS
                [-4.2, -0.5]# LAS_ANGELES
            ], np.zeros((nodes, 1))), axis=1) + RIGHT
        )

        curr = cmob_circuit.get_current_mobjects()
        timer = cmob_circuit.get_timer()

        self.add(curr)
        self.play(timer.animate.set_value(1), run_time=1, rate_func=linear)

        kwhours = Tex("$3.99 * 10^9 kWh$").align_on_border(LEFT).shift(DOWN * 2)
        joules = Tex("$=14.4 * 10^{15} J$").next_to(kwhours, DOWN)
        self.play(timer.animate(rate_func=linear).increment_value(2), Write(kwhours, rate_func=smooth), run_time=2)
        self.play(timer.animate(rate_func=linear).increment_value(1))
        self.play(timer.animate(rate_func=linear).increment_value(2), Write(joules, rate_func=smooth), run_time=2)

        self.wait()

<string>:23: RuntimeWarning: divide by zero encountered in double_scalars                   


In [3]:
%%manim -v WARNING -qm Foolery

class Foolery(Scene):
    def construct(self):
        saturn5_base = SVGMobject("./svgs/saturn5.svg")

        take_off = lambda a: a ** 4
        
        timer = ValueTracker(0)
        take_off_time = 6
        take_off_height = 6
        zoom = ValueTracker(3)
        def position(i):
            return \
                (
                    take_off(max(timer.get_value() - i, 0) / take_off_time) * take_off_height * UP + 
                    RIGHT * (i // 2 * (-1) ** i)
                ) * zoom.get_value()
        def size():
            return zoom.get_value()
        # saturn5 = always_redraw(lambda: saturn5_base.copy().move_to(position(0)).scale(size()))

        other_saturn_5s = VGroup(
            *[
                SVGMobject("./svgs/saturn5.svg")
                for i in range(1, 10)
            ]
        )
        [
            saturn5.add_updater(lambda m: m.move_to(position(i-1)).scale(size()))
            for i, saturn5 in enumerate(other_saturn_5s)
        ]

        self.play(
            Create(other_saturn_5s),
            lag_ratio=0.4
            # *[
            #     anim
            #     for s5 in other_saturn_5s
            #     for anim in fancy(s5)
            # ]
        )
        # self.add(other_saturn_5s)
        # self.play(*fancy(saturn5))
        # self.add(saturn5)

        self.play(
            timer.animate(rate_func=linear).set_value(take_off_time), 
            zoom.animate(**occur_at(1, 2, take_off_time)).set_value(1),

        #     *[
        #         anim 
        #         for s5 in other_saturn_5s 
        #         for anim in fancy(s5, rate_func=occur_at(1, 1, take_off_time)["rate_func"])
        #     ],

        #     run_time=take_off_time
        )
        self.wait()


In [15]:
%%manim -v WARNING -qm Foolery2

class Foolery2(Scene):
    def construct(self):
        saturn5_base = SVGMobject("./svgs/saturn5.svg")

        take_off = lambda a: a ** 4
        
        timer = ValueTracker(0)
        take_off_time = 6
        take_off_height = 6
        zoom = ValueTracker(3)
        def position(i):
            return \
                (
                    take_off(max(timer.get_value() - i, 0) / take_off_time) * take_off_height * UP + 
                    RIGHT * (i // 2 * (-1) ** i)
                ) * zoom.get_value()
        def size():
            return zoom.get_value()

        saturn5s = VGroup(*[
            always_redraw(lambda: saturn5_base.copy().move_to(position(i)).scale(size()))
            for i in range(1, 10)
        ])
        self.play(
            *[DrawBorderThenFill(s5) for s5 in saturn5s],
            run_time=2)
        
        self.wait()



In [14]:
%%manim -v WARNING -ql Foolery3

class Foolery3(Scene):
    def construct(self):
        saturn5_base = SVGMobject("./svgs/saturn5.svg")
        tracker = ValueTracker(0)
        saturn5 = always_redraw(lambda: saturn5_base.copy().move_to(tracker.get_value() * RIGHT))
        self.play(
            Succession(fancy(saturn5),
            tracker.animate.set_value(1))
        )
        self.wait()
